In [1]:
import numpy as np
import pandas as pd
import random as rnd
# import statistics

from itertools import combinations
from collections import Counter
from get_match_data import *

from group_stages import *
from prediction_model import get_model
from simulation import Simulation

In [2]:
YEAR = 2018  # year for the model

In [3]:
rankings, data = get_data(YEAR)
logreg, match_data = get_model(data, report=True)

Mean Validation accuracy: 0.7466066164815621
Test data model accuracy: 0.754321554229769

               precision    recall  f1-score   support

       False       0.78      0.84      0.81      3980
        True       0.71      0.62      0.67      2557

    accuracy                           0.75      6537
   macro avg       0.74      0.73      0.74      6537
weighted avg       0.75      0.75      0.75      6537

Baseline {predict higher ranked team}:
               precision    recall  f1-score   support

       False       0.84      0.69      0.76      3980
        True       0.62      0.79      0.70      2557

    accuracy                           0.73      6537
   macro avg       0.73      0.74      0.73      6537
weighted avg       0.75      0.73      0.73      6537



In [4]:
match_data

,date,team,opponent,team_score,opponent_score,tournament,city,country,neutral,year,team_rank,team_points,opponent_rank,opponent_points,rank_diff,avg_rank,score_diff,team_won,result,comp
589,1998-01-10,Zambia,Malawi,1,0,COSAFA Cup qualification,Blantyre,Malawi,False,1998,54,1616.70,119,1333.92,-65,86.5,1,True,1,False
10,1998-01-10,Malawi,Zambia,0,1,COSAFA Cup qualification,Blantyre,Malawi,False,1998,119,1333.92,54,1616.70,65,86.5,-1,False,-1,False
4,1998-01-18,Lesotho,Zimbabwe,0,2,COSAFA Cup qualification,Maseru,Lesotho,False,1998,152,1159.80,86,1471.12,66,119.0,-2,False,-1,False
681,1998-01-18,Zimbabwe,Lesotho,2,0,COSAFA Cup qualification,Maseru,Lesotho,False,1998,86,1471.12,152,1159.80,-66,119.0,2,True,1,False
393,1998-01-24,Namibia,South Africa,3,2,COSAFA Cup qualification,Windhoek,Namibia,False,1998,112,1364.70,46,1657.97,66,79.0,1,True,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33723,2018-12-06,Philippines,Vietnam,1,2,AFF Championship,Hanoi,Vietnam,False,2018,149,1172.39,128,1277.79,21,138.5,-1,False,-1,False
33718,2018-12-11,Malaysia,Vietnam,2,2,AFF Championship,Kuala Lumpur,Malaysia,False,2018,160,1057.30,128,1277.79,32,144.0,0,False,0,False
34477,2018-12-11,Vietnam,Malaysia,2,2,AFF Championship,Kuala Lumpur,Malaysia,False,2018,128,1277.79,160,1057.30,-32,144.0,0,False,0,False
33720,2018-12-15,Malaysia,Vietnam,0,1,AFF Championship,Hanoi,Vietnam,False,2018,160,1057.30,128,1277.79,32,144.0,-1,False,-1,False


In [5]:
#for y in wc_start:
#    print(y)
#    df = wc_rankings("1998-01-01", wc_start[y])
#    df.to_csv(f"data/rankings{str(y)[-2:]}", index=False)

In [6]:
s = Simulation(YEAR, groups_list[YEAR], logreg, match_data)

In [7]:
import time
tm = time.time()
s.simulate_knockout_round(s.get_knockout_round(False), pr=True)
time.time() - tm

Group Winners: [['Uruguay', 'Russia'], ['Spain', 'Portugal'], ['France', 'Australia'], ['Iceland', 'Argentina'], ['Brazil', 'Switzerland'], ['Germany', 'Sweden'], ['England', 'Belgium'], ['Colombia', 'Poland']]

R16:  ['Uruguay', 'Portugal', 'France', 'Argentina', 'Brazil', 'Sweden', 'England', 'Poland', 'Spain', 'Russia', 'Iceland', 'Australia', 'Germany', 'Switzerland', 'Colombia', 'Belgium']
Uruguay beats Portugal in OT:            [0.21 vs 0.48]
France beats Argentina:                  [0.33 vs 0.32]
Brazil beats Sweden in OT:               [0.76 vs 0.07]
England beats Poland:                    [0.47 vs 0.22]
Spain beats Russia:                      [0.78 vs 0.06]
Iceland beats Australia:                 [0.44 vs 0.25]
Germany beats Switzerland:               [0.65 vs 0.11]
Colombia loses to Belgium:               [0.33 vs 0.33]

QF :  ['Uruguay', 'France', 'Brazil', 'England', 'Spain', 'Iceland', 'Germany', 'Belgium']
Uruguay loses to France:                 [0.2 vs 0.48]
Brazil 

0.3145759105682373

In [8]:
s.get_knockout_round(True)

1-1 Uruguay draws Russia [0.59, 0.26, 0.15] ->  0
5-0 Uruguay beats Saudi Arabia: [0.74, 0.17, 0.09] ->  1
2-0 Uruguay beats Egypt: [0.64, 0.23, 0.13] ->  1
4-0 Russia beats Saudi Arabia: [0.51, 0.28, 0.21] ->  1
3-0 Russia beats Egypt: [0.4, 0.31, 0.29] ->  1
1-0 Saudi Arabia beats Egypt: [0.26, 0.29, 0.45] ->  1
[('Uruguay', 7, 0, 8), ('Russia', 7, 0, 8), ('Saudi Arabia', 3, 0, 1), ('Egypt', 0, 0, 0)] draws: 1

0-3 Spain loses to Portugal [0.4, 0.35, 0.25] ->  -1
2-0 Spain beats Iran: [0.67, 0.23, 0.1] ->  1
1-1 Spain draws Morocco [0.78, 0.15, 0.07] ->  0
1-1 Portugal draws Iran [0.6, 0.26, 0.14] ->  0
4-1 Portugal beats Morocco: [0.71, 0.2, 0.09] ->  1
2-0 Iran beats Morocco: [0.46, 0.3, 0.24] ->  1
[('Portugal', 7, 0, 8), ('Spain', 4, 0, 3), ('Iran', 4, 0, 3), ('Morocco', 1, 0, 2)] draws: 2

0-1 France loses to Denmark [0.53, 0.29, 0.18] ->  -1
2-2 France draws Peru [0.45, 0.33, 0.22] ->  0
5-0 France beats Australia: [0.69, 0.21, 0.1] ->  1
0-1 Denmark loses to Peru [0.27, 0.33, 

[['Uruguay', 'Russia'],
 ['Portugal', 'Spain'],
 ['Peru', 'Denmark'],
 ['Argentina', 'Croatia'],
 ['Switzerland', 'Brazil'],
 ['Germany', 'Mexico'],
 ['Belgium', 'England'],
 ['Poland', 'Colombia']]

In [9]:
def evaluate_tournament(iterations):
    winners_dict = {}
    finalists_dict = {}
    for i in range(iterations):
        gw = s.get_knockout_round(False)
        final_four = s.simulate_knockout_round(gw, pr=False)
        final_four_set = frozenset(final_four)
        if final_four_set in finalists_dict.keys():
            finalists_dict[final_four_set] += 1
        else:
            finalists_dict[final_four_set] = 1

        winner = final_four[0]
        if winner in winners_dict.keys():
            winners_dict[winner] += 1
        else:
            winners_dict[winner] = 1

    print(f"Results for {YEAR} World Cup...")

    print("Most common winners")        
    for team, num in sorted(winners_dict.items(), key=lambda x:x[1], reverse=True)[:10]:  # top 5 most common winners
        print(f"{team}: {round((num / iterations) * 100, 1)}%")

    print("\nMost common final fours")
    for teams, num in sorted(finalists_dict.items(), key=lambda x:x[1], reverse=True)[:10]: # top 5 most commmon final fours
        print(f"{list(teams)}: {round((num / iterations) * 100, 1)}%")

evaluate_tournament(100)

Results for 2018 World Cup...
Most common winners
Brazil: 38.0%
Germany: 28.0%
Spain: 16.0%
France: 6.0%
Portugal: 4.0%
Argentina: 4.0%
Peru: 2.0%
Colombia: 1.0%
England: 1.0%

Most common final fours
['Germany', 'Brazil', 'Spain', 'Portugal']: 7.0%
['Germany', 'Brazil', 'Argentina', 'Portugal']: 4.0%
['Brazil', 'Spain', 'Denmark', 'Germany']: 3.0%
['Brazil', 'Spain', 'Portugal', 'England']: 3.0%
['Germany', 'Belgium', 'Portugal', 'France']: 2.0%
['Brazil', 'Spain', 'England', 'France']: 2.0%
['Brazil', 'Argentina', 'Spain', 'England']: 2.0%
['Germany', 'Brazil', 'Portugal', 'France']: 2.0%
['Germany', 'South Korea', 'Spain', 'France']: 2.0%
['Brazil', 'Iran', 'Spain', 'Germany']: 2.0%


In [10]:
def evaluate_groups(group, iterations):
    winners_dict = {}
    finalists_dict = {}
    for i in range(iterations):
        gw = s.get_knockout_round_group(group, False) #gw = get_knockout_round(groups_list[YEAR], False)
        if gw[0] in winners_dict.keys():
            winners_dict[gw[0]] += 1
        else:
            winners_dict[gw[0]] = 1
            
        gw_set = frozenset(gw)
        if gw_set in finalists_dict.keys():
            finalists_dict[gw_set] += 1
        else:
            finalists_dict[gw_set] = 1


    print(f"Results for {YEAR} World Cup...")

    print("Most common group winners")        
    for team, num in sorted(winners_dict.items(), key=lambda x:x[1], reverse=True):
        print(f"{team}: {round((num / iterations) * 100, 1)}%")

    print("\nMost common final fours")
    for teams, num in sorted(finalists_dict.items(), key=lambda x:x[1], reverse=True):
        print(f"{list(teams)}: {round((num / iterations) * 100, 1)}%")

for group in groups_list[YEAR]:
    print(group)
    evaluate_groups(group, 100)
    print('\n\n')

['Uruguay', 'Russia', 'Saudi Arabia', 'Egypt']
Results for 2018 World Cup...
Most common group winners
Uruguay: 68.0%
Russia: 14.0%
Egypt: 10.0%
Saudi Arabia: 8.0%

Most common final fours
['Uruguay', 'Egypt']: 37.0%
['Russia', 'Uruguay']: 35.0%
['Saudi Arabia', 'Uruguay']: 15.0%
['Russia', 'Egypt']: 6.0%
['Russia', 'Saudi Arabia']: 6.0%
['Saudi Arabia', 'Egypt']: 1.0%



['Spain', 'Portugal', 'Iran', 'Morocco']
Results for 2018 World Cup...
Most common group winners
Spain: 52.0%
Portugal: 37.0%
Iran: 7.0%
Morocco: 4.0%

Most common final fours
['Spain', 'Portugal']: 68.0%
['Morocco', 'Spain']: 10.0%
['Iran', 'Spain']: 10.0%
['Iran', 'Portugal']: 8.0%
['Morocco', 'Portugal']: 4.0%



['France', 'Denmark', 'Peru', 'Australia']
Results for 2018 World Cup...
Most common group winners
France: 55.0%
Peru: 21.0%
Denmark: 19.0%
Australia: 5.0%

Most common final fours
['Peru', 'France']: 37.0%
['Denmark', 'France']: 36.0%
['Peru', 'Denmark']: 11.0%
['Australia', 'France']: 10.0%
['Peru', 'Aus

In [11]:
rankings[rankings["Year"] == YEAR][:15]
# NOTE: 199/900 world cup games end in draw  # 0.2211111111111111
# NOTE: 10084/43752 of all games end in draw. # 0.2304808923020662

,Rank,Team,Score,Year
3961,1,Brazil,2121.91,2018
3962,2,Germany,2099.42,2018
3963,3,Spain,2039.00,2018
3964,4,France,1986.47,2018
3965,5,Argentina,1984.39,2018
3966,6,Portugal,1983.57,2018
3967,7,England,1937.57,2018
3968,8,Colombia,1929.62,2018
3969,9,Belgium,1928.18,2018
3970,10,Italy,1901.35,2018
